Imports and global variable setups

In [11]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 1000
epochs = 20
latent_dim = 256 # what is this?
num_samples = 300000
data_path = "data/names_initials_all_caps.csv"

Read in and vectorize the data

In [12]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[ : min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split(",")
    #Now, use "\t" as the start sequence character and "\n" as the end sequence character
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(seq) for seq in input_texts])
max_decoder_seq_length = max([len(seq) for seq in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),
                              dtype="float32")
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens),
                              dtype="float32")
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens),
                              dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one time step
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            #decoder_target_data is ahead by one time step and doesn't include the start char
            decoder_target_data[i, t-1, target_token_index[char]] = 1

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)


Number of samples: 300000
Number of unique input tokens: 27
Number of unique output tokens 28
Max sequence length for inputs: 23
Max sequence length for outputs: 4
(300000, 23, 27)
(300000, 4, 28)
(300000, 4, 28)


Define an input sequence and process it

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

# Set up the decoder and feed the encoder_states as its initial states
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences, and to return internal states as well.
# We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

Define the model that will turn `encoder_input_data` and `decoder_input_data` into `decoder_target_data`

In [14]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Train and save the model

In [15]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,
          epochs=epochs, validation_split=0.2)
model.save("seq2seq-300000-all-caps.h5") # save model

Train on 240000 samples, validate on 60000 samples
Epoch 1/20
240000/240000 [==============================] - 375s 2ms/step - loss: 1.4153 - acc: 0.3166 - val_loss: 1.1611 - val_acc: 0.3861
Epoch 2/20
240000/240000 [==============================] - 445s 2ms/step - loss: 0.9439 - acc: 0.4524 - val_loss: 0.7713 - val_acc: 0.5133
Epoch 3/20
240000/240000 [==============================] - 596s 2ms/step - loss: 0.7678 - acc: 0.5124 - val_loss: 1.0333 - val_acc: 0.4362
Epoch 4/20
240000/240000 [==============================] - 434s 2ms/step - loss: 0.7412 - acc: 0.5221 - val_loss: 0.7218 - val_acc: 0.5270
Epoch 5/20
240000/240000 [==============================] - 297s 1ms/step - loss: 0.7127 - acc: 0.5365 - val_loss: 0.6782 - val_acc: 0.5496
Epoch 6/20
240000/240000 [==============================] - 296s 1ms/step - loss: 0.6166 - acc: 0.5787 - val_loss: 0.5261 - val_acc: 0.6052
Epoch 7/20
240000/240000 [==============================] - 275s 1ms/step - loss: 0.4074 - acc: 0.6455 - val_

/usr/local/lib/python3.6/site-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_6 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_5/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_5/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '
